In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

# Load and Store  ABS ERP CSV into DataFrame
Load CSV

In [2]:
csv_file = os.path.join('..', 'Resources', 'ABS_ERP_COMP_CLEAN.csv')
# csv_file = "Resources/ABS_ERP_COMP_CLEAN.csv"
erp_data_df = pd.read_csv(csv_file)
erp_data_df.head()

,DATAFLOW,MEASURE: Measure,REGION: Region,FREQ: Frequency,TIME_PERIOD: Time Period,OBS_VALUE,UNIT_MEASURE: Unit of Measure,UNIT_MULT: Unit of Multiplier,OBS_STATUS: Observation Status,OBS_COMMENT: Observation Comment
0,ABS:ERP_COMP_Q(1.0.0),Net Internal Migration,Western Australia,Q: Quarterly,1981-Q2,680.0,NUM: Number,0: Units,NaN,NaN
1,ABS:ERP_COMP_Q(1.0.0),Net Internal Migration,Western Australia,Q: Quarterly,1981-Q3,2002.0,NUM: Number,0: Units,NaN,NaN
2,ABS:ERP_COMP_Q(1.0.0),Net Internal Migration,Western Australia,Q: Quarterly,1981-Q4,441.0,NUM: Number,0: Units,NaN,NaN
3,ABS:ERP_COMP_Q(1.0.0),Net Internal Migration,Western Australia,Q: Quarterly,1982-Q1,494.0,NUM: Number,0: Units,NaN,NaN
4,ABS:ERP_COMP_Q(1.0.0),Net Internal Migration,Western Australia,Q: Quarterly,1982-Q2,621.0,NUM: Number,0: Units,NaN,NaN


TEMP file for INDEX resets for STATEs and QUARTERS

In [ ]:
new_test_region_df = erp_data_df[['REGION: Region']].copy()
new_test_region_df.head()

In [ ]:
new_test_quarter_df = erp_data_df[['TIME_PERIOD: Time Period']].copy()
new_test_quarter_df.head()

In [ ]:
new_test_region_df = new_test_region_df.sort_values('REGION: Region').drop_duplicates('REGION: Region', keep='last')

In [ ]:
new_test_quarter_df = new_test_quarter_df.sort_values('TIME_PERIOD: Time Period').drop_duplicates('TIME_PERIOD: Time Period', keep='last')

In [ ]:
new_test_region_df.reset_index(level=0, inplace=True)
new_test_region_df

In [ ]:
new_test_quarter_df.reset_index(level=0, inplace=True)
new_test_quarter_df

In [ ]:
new_test_region_df.rename(columns = {'index':'state_id', 'REGION: Region': 'state_name'}, inplace=True)
new_test_region_df

In [ ]:
new_test_quarter_df.rename(columns = {'index':'quarter_id', 'TIME_PERIOD: Time Period': 'date_quarter'}, inplace=True)
new_test_quarter_df

---------------------------------------------------------------------------

# Load and Store ABS AWE CSV into DataFrame
Load CSV

In [ ]:
csv_file = os.path.join('..', 'Resources', 'ABS_ERP_COMP_CLEAN.csv')
# csv_file = "Resources/ABS_AWE_CLEAN.csv"
awe_data_df = pd.read_csv(csv_file)
awe_data_df.head()

Check Data Types

In [ ]:
awe_data_df.dtypes

In [ ]:
new_test2_df = awe_data_df[['REGION: Region']].copy()
new_test2_df.head()

In [ ]:
new_test2_df = new_test2_df.sort_values('REGION: Region').drop_duplicates('REGION: Region', keep='last')

In [ ]:
#new_test_df=new_test_df.set_index(name="index",start=100)
new_test2_df.reset_index(level=0, inplace=True)
new_test2_df

In [ ]:
awe_data_df

In [ ]:
# Delete column we don't want
# del awe_data_df['DATAFLOW']
del awe_data_df['OBS_STATUS: Observation Status']
del awe_data_df['OBS_COMMENT: Observation Comment']
del awe_data_df['UNIT_MEASURE: Unit of Measure']
# del awe_data_df['TSEST: Adjustment Type']
# del awe_data_df['SEX: Sex']
del awe_data_df['FREQ: Frequency']
awe_data_df.head()

In [ ]:
# Merge two dataframes using an inner join
awe_merge_df = pd.merge(awe_data_df, new_test2_df, on="REGION: Region")
awe_merge_df

In [ ]:
# Identify incomplete rows
awe_merge_df.count()

In [ ]:
# Identify incomplete rows
awe_data_df.count()

In [ ]:
new_test2_df = awe_data_df[['REGION: Region']].copy()
new_test2_df.head()

In [ ]:
# Rename columns 
awe_data_df = awe_data_df.rename(columns={"MEASURE: Measure": "measure_id","REGION: Region": "region", "TIME_PERIOD: Time Period": "quarter_id",
                                              "OBS_VALUE": "net_change","ESTIMATE_TYPE: Estimate Type":"type_id","SECTOR: Sector":"sector_id","INDUSTRY: Industry":"industry_id"})
awe_data_df

In [ ]:
# Display a statistical overview
# We can infer the maximum allowable individual contribution from 'max'
awe_data_df.describe()

In [ ]:
awe_data_df.to_csv("Resources/johan_awe_employment_clean.csv", index=False)

### Clean DataFrame

In [ ]:
# Identify incomplete rows
awe_data_df.count()

In [ ]:
new_awe_data_df = awe_data_df[['measure_id','state_id','quarter_id','net_change','type_id','sector_id','industry_id']].copy()
new_awe_data_df.head()
                             

In [ ]:
average_aernings_df = awe_data_df[['state_id','quarter_id','net_change']].copy()
average_aernings_df.head()

In [ ]:
#Rename columns 
average_aernings_df = average_aernings_df.rename(columns={"net_change": "avg_earnings"})
average_aernings_df

In [ ]:
new_awe_data_df.to_csv("Resources/johan_awe_employment_clean.csv", index=False)
average_aernings_df.to_csv("Resources/johan_average_earnings_clean.csv", index=False)

### Connect to local database

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = 'Fransien1'
host = 'localhost'
port = 5433
database_name = 'Project2'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

### Check for tables

In [ ]:
insp.get_table_names()

### Use pandas to load csv converted DataFrame into database

In [ ]:
#check if the DF is goof to load
new_test_region_df.head()

In [ ]:
#load STATES DB with teh data from the DF
new_test_region_df.to_sql(name='states', con=engine, if_exists='append', index=False)

In [ ]:
#check if the DF is goof to load
new_test_quarter_df.head()

In [ ]:
#load STATES DB with teh data from the DF
new_test_quarter_df.to_sql(name='quarters', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying
* NOTE: can also check using pgAdmin

In [ ]:
#TEST States DB to see if the dada loaded
pd.read_sql_query('select * from states', con=engine).head()

In [ ]:
#TEST States DB to see if the dada loaded
pd.read_sql_query('select * from quarters', con=engine).head()

### Confirm data has been added by querying 

In [ ]:
pd.read_sql_query('select * from states', con=engine).head()